In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder , OneHotEncoder , StandardScaler
import pickle

In [2]:
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
with open('gender_label_encoder.pkl','rb') as file:
    gender_label_encoder = pickle.load(file)
with open('onehot_encoder_geo.pkl','rb') as file:
    onehot_encoder_geo = pickle.load(file)
with open('churn_modelling_scaler.pkl','rb') as file:
    scaler = pickle.load(file)

In [7]:
data['Gender'] = gender_label_encoder.transform(data['Gender'])

In [14]:
geo_encoded = onehot_encoder_geo.transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded , columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [15]:
data.drop(['Geography'],inplace=True , axis=1)

In [16]:
input_df = pd.concat([data.reset_index(drop=True) , geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [20]:
X = input_df.drop(['EstimatedSalary'],axis=1)
y = input_df['EstimatedSalary']
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.2 , random_state=42)


In [18]:
X_train.shape , y_train.shape

((8000, 12), (8000,))

In [19]:
X_test.shape , y_test.shape

((2000, 12), (2000,))

In [22]:
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

In [23]:
with open('churn_scaler.pkl','wb') as file:
    pickle.dump(ss,file)

In [25]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [27]:
regModel = Sequential([
    Dense(64 , activation='relu' , input_shape = (X_train.shape[1],)),
    Dense(32 , activation='relu'),
    Dense(16 , activation='relu'),
    Dense(8 , activation='relu'),
    Dense(1) # default activation function used named linear activation function
])
regModel.compile(optimizer = 'adam' , loss = 'mean_absolute_error' , metrics = ['mae'])
regModel.summary()

c:\Users\gm782\OneDrive\Desktop\mauravgalik\MLUsingLangchain\ANN_ClassificationProject\classificationEnv\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_1 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,585 (14.00 KB)

 Trainable params: 3,585 (14.00 KB)

 Non-trainable params: 0 (0.00 B)

In [34]:
from tensorflow.keras.callbacks import EarlyStopping , TensorBoard
import datetime
log_dir = "regressionlogs/fit" + datetime.datetime.now().strftime("%Y%M%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir , histogram_freq = 1)
earlystopping_callbacks = EarlyStopping(monitor = 'val_loss' , patience = 20 , restore_best_weights = True)


In [35]:
history = regModel.fit(
    X_train , y_train , validation_data = (X_test , y_test),
    epochs = 100, callbacks = [tensorboard_callback , earlystopping_callbacks]
)

Epoch 1/100


250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 49379.5898 - mae: 49379.5898 - val_loss: 50236.1953 - val_mae: 50236.1953
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 49411.1094 - mae: 49411.1094 - val_loss: 50252.6992 - val_mae: 50252.6992
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 49590.4336 - mae: 49590.4336 - val_loss: 50275.7266 - val_mae: 50275.7266
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 49765.5391 - mae: 49765.5391 - val_loss: 50318.4141 - val_mae: 50318.4141
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 48967.0391 - mae: 48967.0391 - val_loss: 50283.7656 - val_mae: 50283.7656
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 48953.6758 - mae: 48953.6758 - val_loss: 50326.1758 - val_mae: 50326.1758
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 50088.5195 - mae: 50088.5195 - val_loss: 50355.8516 - val_mae: 50355.8516
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 48838.

In [36]:
%load_ext tensorboard

In [38]:
%tensorboard --logdir regressionlogs/fit20250901-150904

In [40]:
test_loss,test_mae = regModel.evaluate(X_test , y_test)
print(f"Test MAE : {test_mae}")
print(f"Test Loss : {test_loss}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 51047.6562 - mae: 51047.6562
Test MAE : 50236.1953125
Test Loss : 50236.1953125


In [41]:
regModel.save("regressionModel.h5")